# GGIbot

This notebook controls execution of GGI using **pyautogui**.

It was written to automate downloading XML files for documents from Insects of Guam I and II
as part of a validation process.

This is a fragile hack which probably runs only on my particular setup.

In [1]:
import pyautogui as pag
import time
import mysecrets
import pandas as pd
import subprocess
#from bs4 import BeautifulSoup
import re
import json
from datetime import datetime

In [4]:
def read_dataset_list():
    '''
    Reads ../dataset-list.md and returns a pandas dataframe
    '''
    
    # Read a markdown file, getting the header from the first row and inex from the second column
    # df = pd.read_table('../dataset-list.md', sep="|", header=0, index_col=1, skipinitialspace=True)
    df = pd.read_table('../dataset-list.md', sep="|", header=0, skipinitialspace=True)

    # Drop the left-most and right-most null columns 
    df = df.dropna(axis=1, how='all')

    # Drop the header underline row
    df = df.iloc[1:]  

    # Left-align strings and column headings
    # df = df.style.set_properties(**{'text-align': 'left'})
    # df = df.set_table_styles([dict(selector = 'th', props=[('text-align', 'left')])])

    # Strip whitespace from end of strings
    df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

    # Strip whitespace from end of column headers
    df.columns = df.columns.str.strip()

    # Drop datasets with no title - we don't need to process these
    df = df.drop(df[df.title == 'no title'].index)
    return df

In [ ]:
XMLDIR = '/home/aubrey/Desktop/data-mining-insects-of-guam/MatCit-Validator'

In [ ]:
pag.PAUSE = 5

In [ ]:
def open_GGI():
    pag.hotkey('ctrl','alt','t')
    pag.typewrite('cd GGI\n')
    pag.typewrite('java -jar GgImagineStarter.jar\n')
    pag.typewrite('\n')
    time.sleep(10)

    pag.click(1141, 565, duration=2) # select configuration
    pag.click(1469, 646, duration=2) # click configuration button
    pag.click(1590, 562, duration=2) # look and feel
    pag.click(1259, 635, duration=2) # select JAVA
    pag.click(1267, 709, duration=2) # OK
    pag.click(1143, 648, duration=2) # OK
    pag.click(1766, 171, duration=2) # maximize window

In [ ]:
def login():
    '''
    This function spoofs a login by clicking on File | Load from GGI server items on the main GGI menu.
    The first time that a download is requested during a session results in a login dialog.
    
    My username is remembered by GGI, so this function provides only my password. Note that I keep 
    this private by reading it from mysecrets.py which is included in .gitignore. The pertinant line looks like
    "password = <my GGI password>".
    
    This function does not download a document from the GGI server, but exits after pressing the "cancel" button.    
    '''
    pag.click(  92,  69, duration=2) # File
    pag.click( 122, 134, duration=2) # load from GGI server
    pag.press('enter')
    pag.typewrite(mysecrets.password)  # password
    pag.press('enter')
    pag.click(1371, 594, duration=2) # cancel

In [ ]:
def load_doc(uuid):
    pag.click(  92,  69, duration=2) # File
    pag.click( 122, 134, duration=2) # load from GGI server
    pag.typewrite(uuid)
    pag.press('enter')
    
# load_doc('FF86FFC39D5F6775A307302F3260421D')

In [ ]:
def export_xml(uuid):
    pag.click(133,  72, duration=2) # Export
    pag.click(151, 154, duration=2) # Export XML
    xml_path = f'{XMLDIR}/{uuid}'
    pag.press('backspace')          # Remove space character in textbox
    pag.typewrite(xml_path)
    pag.press('enter')
    
# export_xml('FF86FFC39D5F6775A307302F3260421D')

In [ ]:
def close_doc():
    pag.click(  92,  69, duration=2) # File
    pag.click( 132, 214, duration=2) # Close Document

# close_doc()

In [ ]:
def process_doc(uuid):
    cl = f'papermill -p UUID {uuid} MatCit-Validator.ipynb {uuid}.ipynb'
    x = subprocess.run(cl, shell=True, capture_output=True, text=True)
    if x.returncode != 0:
        print(x.stderr)
        
# process_doc('FF86FFC39D5F6775A307302F3260421D')

In [2]:
def nth_repl(s, sub, repl, n):
    find = s.find(sub)
    # If find is not -1 we have found at least one match for the substring
    i = find != -1
    # loop util we find the nth or we find no match
    while find != -1 and i != n:
        # find + 1 means we start searching from after the last match
        find = s.find(sub, find + 1)
        i += 1
    # If i is equal to n we found nth match so replace
    if i == n:
        return s[:find] + repl + s[find+len(sub):]
    return s

s = '''
<tr>
    <td>alpha</td>
</tr>
<tr>
    <td>beta</td>
</tr>
<tr>
    <td>cappa</td>
</tr>
'''
print(s)
print()
s = nth_repl(s, '<tr', '<tr caca', 2)
print(s)


<tr>
    <td>alpha</td>
</tr>
<tr>
    <td>beta</td>
</tr>
<tr>
    <td>cappa</td>
</tr>



<tr>
    <td>alpha</td>
</tr>
<tr caca>
    <td>beta</td>
</tr>
<tr>
    <td>cappa</td>
</tr>



In [8]:
def create_status_report():
    

    # TEMP
    
    df = read_dataset_list()
    uuids = list(df.iloc[0:10].uuid.values)
        
    summary_list = []
    for uuid in uuids:
        summary = json.load(open(f'{uuid}_summary.json'))
        summary['uuid'] = uuid
        summary_list.append(summary)
    df_summary = pd.DataFrame(summary_list)
    df_merged = pd.merge(df, df_summary, on='uuid')
    df_merged.drop('status', axis=1, inplace=True)
    #df_merged['validation report'] = f'<a href="{df_merged.uuid}.html">validation report</a>'
    summary_html = df_merged.to_html(index=False)
    
    # Edit so that the table is nicely styled with Bulma
    
    summary_html = summary_html.replace('<table border="1" class="dataframe">', '<table class="table">')
    summary_html = summary_html.replace('<th>', '<th class="has-text-centered">')
    summary_html = summary_html.replace('<td>', '<td class="has-text-centered">')
    
    # Link uuid to validation report
    
    for uuid in uuids:
        summary_html = summary_html.replace(uuid, f'<a href="{uuid}.html">{uuid}</a>')
    
    # Highlight rows which need work
    
    for i, r in df_merged.iterrows():
        if (r['bad collectors'] + r['bad locations'] + r['bad dates']) > 0:
            summary_html = nth_repl(summary_html, '<tr', '<tr class="is-selected"', i+2)  
    
    title_html = f'''
        <p class="title is-1">Insects of Guam Datamining Project</p>
        <p class="subtitle is-3">Status report generated by GGIbot at {datetime.utcnow()} UTC</p>
        <p>Highlited documents failed validation.</p>
        '''

    html = f'''
        <html>
            <header>
                <meta charset="utf-8">
                <meta name="viewport" content="width=device-width, initial-scale=1">
                <title>mat_cit_chk</title>
                <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bulma@0.9.3/css/bulma.min.css">
            </header>
            <body>
                <section class="section">
                    <div class="container">
                        {title_html}
                        {summary_html}
                    </div>
                </section>
            </body>
        </html>        
        '''
    with open('status_report.html', 'w') as f:
        f.write(html) 
        
create_status_report()

In [ ]:
!ls

In [ ]:
%%time

# MAIN

df = read_dataset_list()

# lets process the first 5 datasets

uuids = list(df.iloc[0:10].uuid.values)
open_GGI()
login()
for uuid in uuids:
    load_doc(uuid)
    export_xml(uuid)
    process_doc(uuid)
    close_doc()
create_status_report()
        
print('FINISHED')
print('Please update git repo for ~/Desktop/data-mining-insects-of-guam')
print()